<a href="https://colab.research.google.com/github/odysseus0/tf_learn/blob/master/Keras_Fashion_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import os

%matplotlib inline

tf.VERSION

'1.13.1'

In [0]:
fashion_mnist = keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

In [0]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

## Explore the data

In [0]:
train_images.shape, train_labels.shape, test_images.shape, test_labels.shape

So here we know that just like MNIST, Fashion-MNIST also have image size of 28 x 28.

There are 60000 training examples and 10000 testing examples.

Below is an example of the training data.

In [0]:
plt.imshow(train_images[0])
plt.colorbar()
plt.grid(False)

Now we are going to have a look at the labels.

In [0]:
np.unique(train_labels)

So there are 10 classes of fashion objects in the dataset, just like the 10 digits in the original MNIST.

In [0]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    plt.xlabel(class_names[train_labels[i]])
plt.show()

## Preprocessing

For better training performance, here we are going to first rescale the image pixel range.

In [0]:
train_images = train_images / 255.0
test_images = test_images / 255.0

## Build and train the model

In [0]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])

In [0]:
tpu_model = tf.contrib.tpu.keras_to_tpu_model(
    model,
    strategy=tf.contrib.tpu.TPUDistributionStrategy(
        tf.contrib.cluster_resolver.TPUClusterResolver(
            tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
    )
)
tpu_model.compile(
    optimizer=tf.train.AdamOptimizer(learning_rate=1e-3, ),
    loss=tf.keras.losses.sparse_categorical_crossentropy,
    metrics=['sparse_categorical_accuracy']
)

In [0]:
tpu_model.fit(train_images, train_labels, batch_size=64 * 8, epochs=20, 
          validation_split=0.1)

## CNN from scratch in Keras

In [0]:
model = keras.models.Sequential([
    keras.layers.Reshape((28, 28, 1), input_shape=(28, 28)),
    keras.layers.Conv2D(10, 3, 1, 'same', activation='relu', use_bias=True,
                        kernel_initializer=keras.initializers.he_uniform()),
    keras.layers.Conv2D(10, 3, 1, 'same', activation='relu', use_bias=True,
                        kernel_initializer=keras.initializers.he_uniform()),
    keras.layers.MaxPool2D(2, (2, 2), 'same'),
    keras.layers.Conv2D(40, 3, 1, 'same', activation='relu', use_bias=True,
                        kernel_initializer=keras.initializers.he_uniform()),
    keras.layers.Conv2D(40, 3, 1, 'same', activation='relu', use_bias=True,
                        kernel_initializer=keras.initializers.he_uniform()),
    keras.layers.MaxPool2D(2, (2, 2), 'same'),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])

In [0]:
model.summary()

In [0]:
tpu_model = tf.contrib.tpu.keras_to_tpu_model(
    model,
    strategy=tf.contrib.tpu.TPUDistributionStrategy(
        tf.contrib.cluster_resolver.TPUClusterResolver(
            tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
    )
)
tpu_model.compile(
    optimizer=tf.train.AdamOptimizer(learning_rate=1e-3, ),
    loss=tf.keras.losses.sparse_categorical_crossentropy,
    metrics=['sparse_categorical_accuracy']
)

In [0]:
tpu_model.fit(train_images, train_labels, batch_size=64 * 8, epochs=20, 
          validation_split=0.1)

We got a pretty nice performance boost from using CNN.

##  VGG-16 from Scratch using Keras

Since the dataset is relatively simple in terms of classification difficulty and that it is greyscale image, we are going to reduce the channel depth of the convolution layers to 1/4 of the original in the VGG paper. We would also use only one fully connected layer.

In [0]:
from tensorflow.keras.layers import Reshape, Conv2D, MaxPool2D, Dense, Flatten
from functools import partial

In [0]:
def vgg_conv(kernel_size, num_channel):
  return Conv2D(kernel_size, num_channel, 1, 'same',
                activation='relu',use_bias=True, 
                kernel_initializer=keras.initializers.he_uniform())

vgg_conv_k3 = partial(vgg_conv, 3)
vgg_conv_k1 = partial(vgg_conv, 1)

def vgg_max_pool():
  return MaxPool2D(2, 2, 'same')

def vgg_dense():
  return Dense(4096, 'relu', True)

model = keras.models.Sequential([
    Reshape((28, 28, 1), input_shape=(28, 28)),
    vgg_conv_k3(64),
    vgg_conv_k3(64),
    vgg_max_pool(),
    
    vgg_conv_k3(128),
    vgg_conv_k3(128),
    vgg_max_pool(),
    
    vgg_conv_k3(256),
    vgg_conv_k3(256),
    vgg_conv_k1(256),
    vgg_max_pool(),
    
    vgg_conv_k3(512),
    vgg_conv_k3(512),
    vgg_conv_k1(512),
    vgg_max_pool(),
    
    vgg_conv_k3(512),
    vgg_conv_k3(512),
    vgg_conv_k1(512),
    vgg_max_pool(),
    
    Flatten(),
    vgg_dense(),
    vgg_dense(),
    Dense(10, 'softmax', True)
])

In [34]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_3 (Reshape)          (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 28, 28, 3)         12291     
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 28, 28, 3)         36867     
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 14, 14, 3)         0         
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 14, 14, 3)         147459    
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 14, 14, 3)         147459    
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 7, 7, 3)           0         
__________

In [0]:
tpu_model = tf.contrib.tpu.keras_to_tpu_model(
    model,
    strategy=tf.contrib.tpu.TPUDistributionStrategy(
        tf.contrib.cluster_resolver.TPUClusterResolver(
            tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
    )
)
tpu_model.compile(
    optimizer=tf.train.AdamOptimizer(learning_rate=1e-3, ),
    loss=tf.keras.losses.sparse_categorical_crossentropy,
    metrics=['sparse_categorical_accuracy']
)

In [0]:
tpu_model.fit(train_images, train_labels, batch_size=64 * 8, epochs=20, 
          validation_split=0.1)

## ResNet-18 from Scratch using Keras